In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from pprint import pprint
from mymodel import mymodel
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#iteration is which file it loads, then saves to one higher if training
iteration = 6

In [ ]:
#9 - main version
#10 - linear nn
#11 - linear regression
#12 - logistic regression
version = 9

In [ ]:
#what to do
training = False
printweights = False
runprediction = True
plotting = True

In [ ]:
filename = "weatherCleaned4.csv"
outIdx = 16
inIdx = 16
epochs = 25000
rowDelim = 50000

In [ ]:
#load data from file
data = np.genfromtxt(filename, delimiter=',')[1:]
data = data[:,[2,3,4,5,6,8,11,12,13,14,15,16,17,18,19,20,23]]

In [ ]:
np.random.seed(100)
np.random.shuffle(data)
data = data.astype(float)

In [ ]:
#separate training and testing data
trainy = data[:rowDelim, outIdx]
testy = data[rowDelim:, outIdx]
trainx = data[:rowDelim, :inIdx]
testx = data[rowDelim:, :inIdx]

In [ ]:
#normalize data
mean = trainx.mean(axis=0)
std = trainx.std(axis=0)
trainx -= mean
trainx /= std
testx -= mean
testx /= std

In [ ]:
if printweights:
    print('mean = ')
    pprint(mean)
    print('std = ')
    pprint(std)

In [ ]:
#create model
model = Sequential()

In [ ]:
if version == 9:
    model.add(Dense(12, input_dim = 16, activation='tanh'))
    model.add(Dense(10, input_dim = 12, activation='tanh'))
    model.add(Dense(10, input_dim = 10, activation='tanh'))
    model.add(Dense(1, input_dim = 10, activation='sigmoid'))
if version == 10:
    model.add(Dense(12, input_dim = 16, activation='linear'))
    model.add(Dense(10, input_dim = 12, activation='linear'))
    model.add(Dense(10, input_dim = 10, activation='linear'))
    model.add(Dense(1, input_dim = 10, activation='linear'))
if version == 11:
    model.add(Dense(1, input_dim = 16, activation='linear'))
if version == 12:
    model.add(Dense(1, input_dim = 16, activation='sigmoid'))

In [ ]:
model.compile(loss='mse', optimizer='adam')

In [ ]:
#load previously trained weights
if iteration > 0:
    model.load_weights('./params'+str(version)+'-'+str(iteration))

In [ ]:
#train the model
if training:
    history = model.fit(trainx, trainy, epochs=epochs ,batch_size=100, verbose = 2, validation_data = (testx, testy))
    model.save_weights('./params'+str(version)+'-'+str(iteration+1))

In [ ]:
#print weights and biases for mymodel
if printweights:
    for layer in model.layers:
        print('weights = ')
        pprint(layer.get_weights()[0])
        print('biases = ')
        pprint(layer.get_weights()[1])

In [ ]:
#test the model
if runprediction:
    koutput = np.squeeze(model.predict(testx))
    myoutput = np.squeeze([mymodel(x) for x in testx])
    print("kprediction      actual        myprediction")
    for i in range(20):
        print("%10f    %10f     %10f" % (koutput[i], testy[i], myoutput[i]))
    print('keras mean squared error:  ',np.mean((koutput-testy)**2))
    print('mymodel mean squared error:',np.mean((myoutput-testy)**2))

    if plotting:
        rainy = koutput[np.where(testy==1)]
        notrainy = koutput[np.where(testy==0)]
        plt.figure(figsize=(4,3.5))
        bins = 30
        plt.hist(notrainy, alpha=0.7, bins=bins, label='not rainy',  color='orange', normed=True)
        plt.hist(rainy, alpha=0.5, bins=bins, label='rainy',  color='blue', normed=True)
        plt.show()